In [9]:
import pandas as pd
import numpy as np
import sqlite3
import re
from tqdm import tqdm

dfRNAL = pd.read_csv('NAO MEXE/RNALcomCoordenadas.csv')
dfAirbnb = pd.read_csv('insideairbnb/porto_06-2019_listings.csv')

#incluir apenas concelho do Porto
dfAirbnb = dfAirbnb[dfAirbnb.neighbourhood_group_cleansed == 'PORTO']

print(dfAirbnb.shape)

(8010, 106)


In [10]:
dfAirbnb['license'].value_counts()

Exempt       272
55201        115
53701/AL      27
509067786     24
12345/AL      17
36039/AL      14
30771/AL      13
31971/AL      13
5525/AL       11
26060/AL      11
49461/AL      10
36564/AL      10
26065/AL      10
75330/AL       9
41274/AL       9
73055/AL       9
5929/AL        9
39504/AL       8
83972AL        8
55224/AL       8
26904/AL       8
45802/AL       8
71790/AL       8
70985/AL       8
44984/AL       8
75557/AL       7
31936/AL       7
60599          7
47348/AL       7
53975/AL       7
            ... 
86954/AL       1
47346/AL       1
84178/AL       1
89323/AL       1
78075/AL       1
51218/AL       1
29952/AL       1
60694/AL       1
67073/AL       1
70340/AL       1
66490/AL       1
96316/AL       1
93466AL        1
66405          1
59151/AL       1
54128/AL       1
81384/AL       1
98099/AL       1
68507/AL       1
27017/AL       1
38964/AL       1
82556/AL       1
86460/AL       1
61694/AL       1
67412/AL       1
79793/AL       1
54424/AL       1
6816/AL       

In [11]:
def sonumeros(a):
    return re.sub('[^0-9]','', a)

def semano(a):
    return re.sub('\/.*','',str(a))

#Uniformiza numero de registo, remove duplicados (quartos do mesmo estabelecimento), inclui apenas anuncios com alguma review em 2019 ...
dfAirbnb.license = dfAirbnb.license.apply(semano)
dfAirbnb.license = dfAirbnb.license.apply(sonumeros)
dfAirbnb.license.replace('', np.nan, inplace=True)
dfAirbnb.dropna(subset=['first_review'], inplace=True)
print(dfAirbnb.shape)
dfAirbnb['first_review']=pd.to_datetime(dfAirbnb['first_review'], format='%Y-%m-%d')
dfAirbnb['last_review']=pd.to_datetime(dfAirbnb['last_review'], format='%Y-%m-%d')
dfAirbnb = dfAirbnb[(dfAirbnb['last_review'] > '2019-01-01')]
dfAirbnb.dropna(subset=['license'],inplace=True)

dfRNAL.Nregisto = dfRNAL.Nregisto.apply(semano)

print(dfAirbnb.shape)
print(dfRNAL.shape)

dfRNAL.head()

(6951, 106)
(5115, 106)
(8317, 28)


,Unnamed: 0,Nregisto,Data do registo,Nome do Alojamento,Imovel Posterior 1951,Data Abertura Publico,Modalidade,N Camas,N Utentes,N Quartos,...,Nome do Titular da Exploração,Titular Qualidade,Contribuinte,Titular Tipo,Titular Pais,Contacto (Telefone),Contacto (Fax),Contacto (Telemovel),LATITUDE,LONGITUDE
0,0,96492,2019-05-21,A Moment in Time - 3,S,2019-05-07,Apartamento,1,2,1.0,...,"Barbimoda - Importação e Exportação, Lda",Proprietário,502028742.0,Pessoa coletiva,Portugal,NaN,NaN,NaN,41.150099,-8.612137
1,1,25605,2016-03-08,MOUZINHO DA SILVEIRA 2D,N,2017-09-01,Apartamento,2,4,1.0,...,NaN,Arrendatário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.142977,-8.614052
2,2,22983,2015-12-23,Suites Cocorico,N,2016-02-04,Estabelecimento de hospedagem,5,5,5.0,...,"C La Luz , Lda",Proprietário,514231181.0,Pessoa coletiva,Portugal,917510464,NaN,912775187,41.144298,-8.604996
3,3,63738,2018-02-02,"CASA DO BAIRRO 2""""",N,2018-02-02,Apartamento,2,4,1.0,...,NaN,Arrendatário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.157944,-8.629105
4,4,17775,2015-07-20,0,N,2015-07-20,Moradia,4,6,3.0,...,NaN,Proprietário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.145303,-8.613585


In [12]:
dfJoin = dfAirbnb.set_index('license').join(dfRNAL.set_index('Nregisto'))
dfJoin.index.name = 'REGISTO'

dfJoin.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,Nome do Titular da Exploração,Titular Qualidade,Contribuinte,Titular Tipo,Titular Pais,Contacto (Telefone),Contacto (Fax),Contacto (Telemovel),LATITUDE,LONGITUDE
REGISTO,,,,,,,,,,,,,,,,,,,,,
00000,27712756,https://www.airbnb.com/rooms/27712756,20190625184202,2019-06-25,Porto4You,"O apartamento está localizado no Porto, com pa...",NaN,"O apartamento está localizado no Porto, com pa...",none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100252,3681924,https://www.airbnb.com/rooms/3681924,20190625184202,2019-06-25,"New, cozy flat with garage, host 6","A totally renewed apartment with 3 bedrooms, ...",You will have here two double beds and 2 singl...,"A totally renewed apartment with 3 bedrooms, ...",none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10052,3790726,https://www.airbnb.com/rooms/3790726,20190625184202,2019-06-25,Charming & Unique City Flat,Located in Porto downtown in a building renova...,"Centrally located in a quiet square, the flat ...",Located in Porto downtown in a building renova...,none,"Overlooking the Douro River, Porto is one of t...",...,NaN,Proprietário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.152758,-8.608035
10231,6029395,https://www.airbnb.com/rooms/6029395,20190625184202,2019-06-25,Cosy flat at 5min from center Porto,"Apartamento acolhedor, com uma localização pri...",NaN,"Apartamento acolhedor, com uma localização pri...",none,A melhor atração da rua é sem duvida a Casa da...,...,NaN,Proprietário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.160773,-8.631677
10248,5465136,https://www.airbnb.com/rooms/5465136,20190625184202,2019-06-25,Casa de Salgueiros,"Cozy apartment with 4 bedrooms, for 8 people. ...",Cozy apartment with 4 rooms for up to 8 person...,"Cozy apartment with 4 bedrooms, for 8 people. ...",none,"Quiet neighborhood, with easy access to a lot ...",...,NaN,Proprietário,NaN,Pessoa singular (empresário em nome individual),Portugal,NaN,NaN,NaN,41.170034,-8.597130


In [13]:
dfJoin[['first_review', 'Data Abertura Publico']]

,first_review,Data Abertura Publico
REGISTO,,
00000,2018-09-02,NaN
100252,2014-08-07,NaN
10052,2014-08-13,2014-09-29
10231,2015-06-07,2015-03-26
10248,2015-04-05,2015-03-26
10484,2015-02-28,2015-03-30
10845,2015-04-22,2015-04-10
10847,2016-06-14,2015-04-02
10864,2014-09-18,2015-04-06


In [14]:
dfJoin.to_csv('joined_output_price.csv')



In [15]:
cartoColumns = ['REGISTO', 'listing_url', 'price', 'LATITUDE','LONGITUDE']

dfCarto = dfJoin

dfCarto.reset_index(level=0, inplace=True)

print(dfCarto.shape)

dfCarto = dfCarto.sort_values('first_review')

dfCarto = dfCarto.drop_duplicates(subset = ['REGISTO'], keep='first')

dfCarto[['REGISTO', 'first_review']]

dfCarto.to_csv('carto1.csv', columns = cartoColumns)

#print(dfCarto.shape)


(5115, 133)
